In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('marketing_campaign_data_cleaned.csv')
print("Dataset loaded successfully!")

df.head()

Dataset loaded successfully!


,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date
0,1,Innovate Industries,Email,Men 18-24,30,Google Ads,0.04,16174.0,6.29,Chicago,Spanish,506,1922,6,Health & Wellness,2021-01-01
1,2,NexGen Systems,Email,Women 35-44,60,Google Ads,0.12,11566.0,5.61,New York,German,116,7523,7,Fashionistas,2021-01-02
2,3,Alpha Innovations,Influencer,Men 25-34,30,YouTube,0.07,10200.0,7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,2021-01-03
3,4,DataTech Solutions,Display,All Ages,60,YouTube,0.11,12724.0,5.55,Miami,Mandarin,217,1820,7,Health & Wellness,2021-01-04
4,5,NexGen Systems,Email,Men 25-34,15,YouTube,0.05,16452.0,6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,2021-01-05


In [2]:
# Extract components the year, month, or day of the week from the Date column.

# Convert Date to datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Extract Year
df['Year'] = df['Date'].dt.year

# Extract Month
df['Month'] = df['Date'].dt.month
df['Month_Name'] = df['Date'].dt.strftime('%B') # %B gives full month name

# Extract Day of Week
df['Day_of_Week'] = df['Date'].dt.day_name()

# Verify the new columns
print("\n--- First 5 rows with new time features ---")
df.head()


--- First 5 rows with new time features ---


,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,Year,Month,Month_Name,Day_of_Week
0,1,Innovate Industries,Email,Men 18-24,30,Google Ads,0.04,16174.0,6.29,Chicago,Spanish,506,1922,6,Health & Wellness,2021-01-01,2021,1,January,Friday
1,2,NexGen Systems,Email,Women 35-44,60,Google Ads,0.12,11566.0,5.61,New York,German,116,7523,7,Fashionistas,2021-01-02,2021,1,January,Saturday
2,3,Alpha Innovations,Influencer,Men 25-34,30,YouTube,0.07,10200.0,7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,2021-01-03,2021,1,January,Sunday
3,4,DataTech Solutions,Display,All Ages,60,YouTube,0.11,12724.0,5.55,Miami,Mandarin,217,1820,7,Health & Wellness,2021-01-04,2021,1,January,Monday
4,5,NexGen Systems,Email,Men 25-34,15,YouTube,0.05,16452.0,6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,2021-01-05,2021,1,January,Tuesday


**We have Conversion_Rate and ROI, we can calculate other metrics to understand campaign efficiency and effectiveness:**
- **Cost Per Click (CPC)**: Helps understand how much you're paying for each click. Lower CPC generally means more efficient ad spend for engagement.

- **Cost Per Acquisition (CPA)**: This is a crucial metric for understanding the cost-effectiveness of converting a customer. It tells us how much it costs to acquire one conversion.

**We have 'Clicks' and 'Impressions', we can calculate CLick Through Rate ratio (CTR):**
- **Click-Through Rate (CTR)**: Indicates how engaging or relevant an ad is to its audience. A higher CTR often means targeting are effective in getting people to interact.

**We have 'Acquisition_Cost' and 'ROI', we can estimate profit generated by a campaign:**
- **Estimated_Profit**: useful when making budget allocation recommendations.


In [3]:
# Calculate Cost_Per_Click (CPC)
# Formula: CPC = Acquisition_Cost / Clicks
# Handle potential division by zero: set it to 0
df['Cost_Per_Click'] = df['Acquisition_Cost'] / df['Clicks']
df['Cost_Per_Click'] = df['Cost_Per_Click'].replace([np.inf, -np.inf], 0) # Replace inf with 0

# Calculate Click-Through Rate (CTR)
# Formula: CTR = (Clicks / Impressions) * 100 (for percentage)
# Handle potential division by zero: Set it to 0
df['Click_Through_Rate'] = df.apply(lambda row: (row['Clicks'] / row['Impressions']) * 100 if row['Impressions'] > 0 else 0, axis=1)

# Calculate Estimated Conversions
# Formula: Estimated Conversions = Impressions * Conversion_Rate
df['Estimated_Conversions'] = df['Impressions'] * df['Conversion_Rate']
df['Estimated_Conversions'] = df['Estimated_Conversions'].round().astype(int)


# Calculate Cost Per Acquisition (CPA)
# FormulaL CPA = Acquisition_Cost / Estimated_Conversions
df['Cost_Per_Acquisition'] = df.apply(lambda row: row['Acquisition_Cost'] / row['Estimated_Conversions'] if row['Estimated_Conversions'] > 0 else 0, axis=1)

# Calculate Estimated Profit
# Formula: Profit = ROI * Acquisition_Cost (ROI = (Net Profit / Acquisition_Cost))
df['Estimated_Profit'] = df['ROI'] * df['Acquisition_Cost']

# Verify the new columns and review descriptive statistics
print("\n--- Summary statistics for new performance metrics ---")
print(df[['Cost_Per_Click', 'Click_Through_Rate', 'Estimated_Conversions', 'Cost_Per_Acquisition', 'Estimated_Profit']].describe())

# Check for any new NaNs introduced by divisions
print("\n--- Missing values count after new feature creation ---")
print(df.isnull().sum())




--- Summary statistics for new performance metrics ---
       Cost_Per_Click  Click_Through_Rate  Estimated_Conversions  \
count   200000.000000       200000.000000          200000.000000   
mean        32.008490           14.040550             440.669940   
std         26.926121           13.088122             322.707873   
min          5.021084            1.005429              10.000000   
25%         15.091967            5.860555             178.000000   
50%         22.774008            9.978933             359.000000   
75%         38.599088           16.969863             642.000000   
max        199.960000           99.202393            1500.000000   

       Cost_Per_Acquisition  Estimated_Profit  
count         200000.000000     200000.000000  
mean              63.320608      62586.943371  
std               96.724906      31667.568417  
min                3.343792      10064.070000  
25%               18.034114      37809.710000  
50%               33.316339      56193.5250

**Summary Statistics for New Metrics:**

- **Cost_Per_Click:** Ranges from ~$5 to ~$200. The mean is ~$32. The spread is plausible.

- **Click_Through_Rate:** Ranges from ~1% to ~99%. The mean is ~14%. Reasonable for digital campaigns, with some highly effective ones.

- **Estimated_Conversions:** Ranges from 10 to 1500, with a mean of ~440.

- **Cost_Per_Acquisition:** Ranges from ~$3.34 to ~$1991.60, with a mean of ~$63.32. Some campaigns are highly efficient in acquiring conversions, while others are very costly. A key insight to explore.

- **Estimated_Profit:** Ranges from ~$10,064 to ~$159,456, with a mean of ~$62,586. A reasonable range for campaign profitability.

**Extract Gender, and Age from Target Audience**

In [11]:
# Extract Gender
def extract_gender(audience):
    s_audience = str(audience)
    if 'Men' in s_audience:
        return 'Male'
    elif 'Women' in s_audience:
        return 'Female'
    else:
        return 'Unspecified/All' # For 'All Ages' or any other non-gendered audience

df['Gender_Target'] = df['Target_Audience'].apply(extract_gender)
print(f"Unique values for 'Gender_Target': {df['Gender_Target'].unique()}")

# Extract Age
import re 
def extract_age_group(audience):
    s_audience = str(audience) # Ensure it's a string
    age_match = re.search(r'(\d{2}-\d{2})', s_audience) # Looks for 'XX-YY' pattern
    if age_match:
        return age_match.group(1)
    elif 'All Ages' in s_audience:
        return 'All Ages'
    else:
        return 'Unspecified' # For any audience not matching typical age patterns

df['Age_Group_Target'] = df['Target_Audience'].apply(extract_age_group)
print(f"Unique values for 'Age_Group_Target': {df['Age_Group_Target'].unique()}")


print("\n--- Value Counts for new Target Audience features ---")
print(df['Gender_Target'].value_counts())
print(df['Age_Group_Target'].value_counts())

Unique values for 'Gender_Target': ['Male' 'Female' 'Unspecified/All']
Unique values for 'Age_Group_Target': ['18-24' '35-44' '25-34' 'All Ages']

--- Value Counts for new Target Audience features ---
Gender_Target
Male               80281
Female             79700
Unspecified/All    40019
Name: count, dtype: int64
Age_Group_Target
25-34       80036
18-24       40258
All Ages    40019
35-44       39687
Name: count, dtype: int64


In [12]:
# Save dataset
output_file_path_enriched = 'marketing_campaign_data_enriched.csv'
df.to_csv(output_file_path_enriched, index=False)
print(f"Enriched dataset saved successfully to: {output_file_path_enriched}")

Enriched dataset saved successfully to: marketing_campaign_data_enriched.csv
